In [128]:
import pandas as pd
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.data import Dataset
#from torchvision import Datasets
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import random

In [3]:
data=pd.read_csv('engfrench.csv')

In [4]:
print(len(data))

175621


In [5]:
data.iloc[170000]

English words/sentences    The guards found a hacksaw blade in the prison...
French words/sentences     Les gardiens trouvèrent une lame de scie à mét...
Name: 170000, dtype: object

In [6]:
from transformers import AutoTokenizer

In [7]:
tokenizer = AutoTokenizer.from_pretrained("Cohere/multilingual-22-12")

tokenizer_config.json:   0%|          | 0.00/428 [00:00<?, ?B/s]

C:\Users\maddo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\maddo\.cache\huggingface\hub\models--Cohere--multilingual-22-12. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/10.6M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [8]:
PAD_TOKEN_ID=0

In [9]:
tokenizer.add_special_tokens({'additional_special_tokens': [' _ENFR_ ']})

1

In [113]:
SOS_TOKEN_ID = tokenizer.cls_token_id if tokenizer.cls_token_id is not None else 0
EOS_TOKEN_ID = tokenizer.sep_token_id if tokenizer.sep_token_id is not None else 0

In [117]:
VOCAB_SIZE = 501160

In [10]:
txten=data.iloc[0,0]
txtfr=data.iloc[0,1]


In [11]:
txten

'Hi.'

In [12]:
txtfr

'Salut!'

In [13]:
token=tokenizer.tokenize(txten)

In [14]:
token

['Hi', '.']

In [15]:
tokenfr=tokenizer.tokenize(txtfr)

In [16]:
tokenfr

['Salut', '!']

In [17]:
len(token)

2

In [18]:
len(tokenfr)

2

In [19]:

dat=txten+' _ENFR_ '+txtfr

In [20]:
dat

'Hi. _ENFR_ Salut!'

In [21]:
tokdat=(tokenizer.tokenize(dat))


In [22]:
print(tokdat)

['Hi', '.', ' _ENFR_ ', 'Salut', '!']


In [23]:
tokdat=tokenizer.convert_tokens_to_ids(tokdat)

In [24]:
for i in range(len(tokdat)):
    print(tokdat[:i],tokdat[i])

[] 22293
[22293] 119
[22293, 119] 501153
[22293, 119, 501153] 83440
[22293, 119, 501153, 83440] 106


In [44]:
class dataset(Dataset):
    def __init__(self,x):
        self.x=x
        
    def __len__(self):
        return len(self.x)
    def __getitem__(self,i):
        inp=self.x
        return torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,0]))),torch.tensor(tokenizer.convert_tokens_to_ids(tokenizer.tokenize(inp.iloc[i,1])))
        



In [45]:
datanewdataset=dataset(data)

In [46]:
tokenizer.decode(datanewdataset[2000][1])

"Ne t ' en va pas!"

In [47]:
tokenizer.decode(datanewdataset[2000][0])

"Don ' t leave!"

In [48]:
def custom_collate_fn(batch):
    # 'batch' is a list of tuples: [(input_tensor_1, target_tensor_1), (input_tensor_2, target_tensor_2), ...]

    # 1. Separate inputs and targets
    inputs = [torch.tensor(item[0]) for item in batch]
    targets = [torch.tensor(item[1]) for item in batch]

    # 2. Pad the inputs and targets to the length of the longest sequence in the batch
    # batch_first=True makes the output shape (BatchSize, SequenceLength)
    inputs_padded = pad_sequence(inputs, batch_first=True, padding_value=PAD_TOKEN_ID)
    targets_padded = pad_sequence(targets, batch_first=True, padding_value=PAD_TOKEN_ID)
    
    return inputs_padded, targets_padded

In [49]:
haha=DataLoader(datanewdataset,batch_size=32,shuffle=True,collate_fn=custom_collate_fn)

In [50]:
embedsize=len(tokenizer.get_vocab())

In [51]:
print(embedsize)

501154


In [52]:
for i,x in enumerate(haha):
    print(tokenizer.decode(x[1][0]))
    print(tokenizer.decode(x[0][0]))
    break

Y a - t - il un endroit en Europe que tu veuilles visiter? [PAD]
Is there somewhere in Europe you ' d like to visit?


C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


In [67]:
xa=torch.randn(10,100)

In [68]:
ha=nn.LSTM(100,200)

In [70]:
ya=ha(xa)

In [74]:
ya[0].shape

torch.Size([10, 200])

In [78]:
ya[1][0].shape

torch.Size([1, 200])

In [79]:
ya[1][1].shape

torch.Size([1, 200])

In [101]:
class enc(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb=nn.Embedding(501160,10)
        self.lstm=nn.LSTM(input_size=10,hidden_size=100,num_layers=3,batch_first=True,bidirectional=False)
        self.rel=nn.ReLU()
    def forward(self,x):
        embx=self.emb(x)
        out,(hid,cell)=self.lstm(embx)
        return hid,cell
        

        
        

In [122]:
class dec(nn.Module):
    def __init__(self):
        super().__init__()
        self.emb = nn.Embedding(VOCAB_SIZE, 10)
        self.lstm = nn.LSTM(input_size=10, hidden_size=100, num_layers=3, batch_first=True, bidirectional=False)
        self.fc_out = nn.Linear(100, VOCAB_SIZE) # Map hidden state to Vocab Size
        
    def forward(self, x, hidden, cell):
        # x shape: [batch_size] -> unsqueeze to [batch_size, 1]
        x = x.unsqueeze(1)
        embx = self.emb(x)
        
        output, (hidden, cell) = self.lstm(embx, (hidden, cell))
        
        # prediction shape: [batch_size, VOCAB_SIZE]
        prediction = self.fc_out(output.squeeze(1))
        return prediction, hidden, cell

In [123]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        # source: [batch, src_len]
        # target: [batch, trg_len]
        
        batch_size = source.shape[0]
        target_len = target.shape[1]
        vocab_size = VOCAB_SIZE
        
        outputs = torch.zeros(batch_size, target_len, vocab_size).to(self.device)
        
        # 1. Encode
        hidden, cell = self.encoder(source)
        
        # 2. First input to decoder (Start Token)
        # We use the SOS_TOKEN_ID defined at the top
        input_token = torch.full((batch_size,), SOS_TOKEN_ID, dtype=torch.long).to(self.device)
        
        # 3. Decode
        for t in range(target_len):
            output, hidden, cell = self.decoder(input_token, hidden, cell)
            outputs[:, t, :] = output
            
            # Teacher forcing
            top1 = output.argmax(1)
            use_teacher = random.random() < teacher_forcing_ratio
            
            # If using teacher forcing, next input is actual target token
            # Else, next input is predicted token
            if t < target_len - 1:
                input_token = target[:, t] if use_teacher else top1
            
        return outputs

In [124]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

encoder = enc()
decoder = dec()
model = Seq2Seq(encoder, decoder, device).to(device)

# Optimization
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_TOKEN_ID)

Using device: cuda


In [125]:
def train_one_epoch(model, iterator, optimizer, criterion, clip=1):
    model.train()
    epoch_loss = 0
    
    for i, (src, trg) in enumerate(iterator):
        src = src.to(device)
        trg = trg.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        output = model(src, trg)
        
        # Reshape for Loss
        # output: [batch, len, vocab] -> [batch*len, vocab]
        # trg: [batch, len] -> [batch*len]
        output_dim = output.shape[-1]
        output = output.view(-1, output_dim)
        trg = trg.view(-1)
        
        loss = criterion(output, trg)
        loss.backward()
        
        # Clip gradients to prevent explosion
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        
        epoch_loss += loss.item()
        
        if i % 10 == 0:
            print(f"Batch {i}, Loss: {loss.item()}")
            
    return epoch_loss / len(iterator)

In [126]:
def predict_translation(model, sentence, max_len=20):
    model.eval()
    
    # Tokenize input
    tokens = tokenizer.tokenize(sentence)
    ids = tokenizer.convert_tokens_to_ids(tokens)
    src_tensor = torch.tensor(ids, dtype=torch.long).unsqueeze(0).to(device) # [1, seq_len]
    
    with torch.no_grad():
        hidden, cell = model.encoder(src_tensor)
    
    # Start with SOS token
    inputs = [SOS_TOKEN_ID]
    
    for _ in range(max_len):
        input_tensor = torch.tensor([inputs[-1]], dtype=torch.long).to(device)
        
        with torch.no_grad():
            output, hidden, cell = model.decoder(input_tensor, hidden, cell)
            
        pred_token = output.argmax(1).item()
        inputs.append(pred_token)
        
        # Stop if EOS or padding is predicted
        if pred_token == EOS_TOKEN_ID or pred_token == PAD_TOKEN_ID:
            break
            
    # Convert IDs back to tokens
    predicted_tokens = tokenizer.convert_ids_to_tokens(inputs[1:]) # Skip SOS
    return tokenizer.convert_tokens_to_string(predicted_tokens)

In [129]:
# Example run (Uncomment to train)
print("Starting Training...")
loss = train_one_epoch(model, haha, optimizer, criterion)
print(f"Epoch Loss: {loss}")

Starting Training...


C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  inputs = [torch.tensor(item[0]) for item in batch]
C:\Users\maddo\AppData\Local\Temp\ipykernel_24132\1408414250.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  targets = [torch.tensor(item[1]) for item in batch]


Batch 0, Loss: 13.119987487792969
Batch 10, Loss: 12.9395751953125
Batch 20, Loss: 11.217743873596191
Batch 30, Loss: 9.204442024230957
Batch 40, Loss: 7.543518543243408
Batch 50, Loss: 6.7631988525390625
Batch 60, Loss: 6.942978858947754
Batch 70, Loss: 6.657811641693115
Batch 80, Loss: 6.904018878936768
Batch 90, Loss: 6.449203014373779
Batch 100, Loss: 6.499974250793457
Batch 110, Loss: 6.70802116394043
Batch 120, Loss: 6.178584575653076
Batch 130, Loss: 6.21954870223999
Batch 140, Loss: 6.309323310852051
Batch 150, Loss: 6.672277450561523
Batch 160, Loss: 6.6075568199157715
Batch 170, Loss: 6.3702521324157715
Batch 180, Loss: 6.683192729949951
Batch 190, Loss: 6.3405327796936035
Batch 200, Loss: 6.370121955871582
Batch 210, Loss: 6.286919116973877
Batch 220, Loss: 6.384129524230957
Batch 230, Loss: 6.0717997550964355
Batch 240, Loss: 6.195898532867432
Batch 250, Loss: 6.632014274597168
Batch 260, Loss: 6.346672058105469
Batch 270, Loss: 6.0875935554504395
Batch 280, Loss: 6.1714339

KeyboardInterrupt: 

In [132]:
translation = predict_translation(model, "Hello HI")
print(f"Translation: {translation}")

Translation: '...................
